# 使用keras实现卷积神经网络
这里仍然使用mnist数据   
通过tf.keras.Model来自定义网络结构

In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

In [14]:
class MNistLoader():
    def __init__(self):
        # 定义mnist对象
        mnist = tf.keras.datasets.mnist
        # 读取mnist数据
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # mnist的数据为 28*28的0~255的值, 需要将图像转化为[张数， 宽度， 高度， 通道]， 并归一化
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1)

        self.train_label = self.train_label.astype(np.float32)
        self.test_label = self.test_label.astype(np.float32)

        # 统计相关信息
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从train_data中随机抽出batch_size个数据
        index = np.random.randint(0, self.num_train_data, batch_size)
        # np数组可以接受list索引，返回list索引对应的值, 原生数据不支持
        return self.train_data[index, :], self.train_label[index]
        


## 定义CNN模型
和MLP相比，只是新加入了卷积层和池化层，这里的网络结构并不唯一，可以增加、删除、调整CNN的网络结构和参数，以达到更好的效果

In [15]:
class CNN(tf.keras.Model):
    def __init__(self):
        # 继承父类的属性和方法
        super().__init__()
        # 定义网络层
        self.conv1 = layers.Conv2D(
            # 卷积层神经元（卷积核）数目, 默认使用kernel_initializer默认使用glorot_uniform进行初始化，简单来讲就是有32个不一样的卷积核，可以指定kernel_initializer
            filters=32,
            # 感受野大小
            kernel_size=[5, 5],
            # padding策略, same/vaild, same表示图片卷积后大小不变，valid表示卷积后图像减小（不对标远进行填充）
            padding='same',
            # 激活函数
            activation=tf.nn.relu
        )

        self.pool1 = layers.MaxPool2D(
            pool_size=[2,2],
            strides=2
        )

        self.conv2 = layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding="same",
            activation=tf.nn.relu
        )

        self.pool2 = layers.MaxPool2D(pool_size=[2,2], strides=2)
        # TODO, 和Flatten的区别是什么
        self.flatten = layers.Reshape(target_shape=(7*7*64, ))
        self.dense1 = layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

## 定义超参和实例化对象


In [16]:
# 定义超参
batch_size = 50
epoch = 5
learning_rate = 0.001

# 实例化数据和模型对象
mnist = MNistLoader()
cnn = CNN()

# 
optimizer = optimizers.Adam(learning_rate=learning_rate)

## 训练过程

In [17]:
# num_batches 
num_batches = int(mnist.num_train_data*epoch//batch_size)
# 循环喂数据
for batch_index in range(num_batches):
    X, y = mnist.get_batch(batch_size=batch_size)
    # 记录梯度信息
    with tf.GradientTape() as tape:
        # 默认调用call方法
        y_pred = cnn(X)
        # 计算Loss
        loss = tf.losses.sparse_categorical_crossentropy(y_pred=y_pred, y_true=y)
        # 每个类别的误差平均值
        loss = tf.reduce_mean(loss)
        if batch_index % 500 == 0:
            print("batch %d \t loss: %f" %(batch_index, loss))
    # 梯度更新
    grads = tape.gradient(loss, cnn.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, cnn.variables))


batch 0 	 loss: 2.305305
batch 500 	 loss: 0.015411
batch 1000 	 loss: 0.003332
batch 1500 	 loss: 0.003056
batch 2000 	 loss: 0.044779
batch 2500 	 loss: 0.002426
batch 3000 	 loss: 0.002162
batch 3500 	 loss: 0.035119
batch 4000 	 loss: 0.006237
batch 4500 	 loss: 0.001408
batch 5000 	 loss: 0.029309
batch 5500 	 loss: 0.001738


## 验证指标
这里还是使用sparse_acc来进行

In [22]:
# # 定义指标对象
# sparse_acc = tf.keras.metrics.SparseCategoricalCrossentropy()

# # 使用测试集进行验证
# batch_num = int(mnist.num_test_data / batch_size)
# for batch_index in range(batch_num):
#     start_index, end_index = batch_index*batch_size, (batch_index+1)*batch_size
#     y_pred = cnn.predict(mnist.test_data[start_index:end_index])
#     # 更新
#     # sparse_acc , y_true是数值， Y_pred为概率，如果y_true对应位置的值为y_pred对应的最大值的索引，则预测正确
#     # 如 y_true = [[2, 1]] y_pred=[[0.5, 0.4, 0.1], [0.9, 0.05, 0.05]] 则 acc=1/2
#     sparse_acc.update_state(mnist.test_label[start_index:end_index], y_pred)
# print(sparse_acc.weights)
# print("test_acc: %f" % sparse_acc.result())

def acc():
    # 定义检测指标对象
    sparse_acc = tf.keras.metrics.SparseCategoricalAccuracy()
    batch_num= int(mnist.num_test_data//batch_size)

    for batch_index in range(batch_num):
        start_index, end_index = batch_index * batch_size, (batch_index+1)*batch_size
        y_pred = cnn.predict(mnist.test_data[start_index: end_index])
        sparse_acc.update_state(mnist.test_label[start_index:end_index], y_pred)
    print("test_acc: %f" % sparse_acc.result())
acc()


test_acc: 0.991600


可以发现，相对与MLP，准确率有很大的提升.  
事实上，通过改变网络结构（比如加入dropout层防止过拟合）准确率还有进一步的提高

## 使用keras中预定义的经典卷积神经网络结构

tf.keras.applications中有一些预定义号的经典卷积神经网络结构,  
如VGG16、 VGG19、ResNet、MobelNet等。  
可以直接调用这些经典的卷积网络，甚至可以加载于训练的参数，而无需手动定义网络结构

In [23]:
# 获取模型参数
model = tf.keras.applications.MobileNetV2()

14540800/14536120 [==============================] - 39s 3us/step


当执行上述代码时，tf会自动下载预训练的参数。  
也可以通过设置weights参数为None来随机初始化变量。  

每个网络都有自己的详细参数设置，一些共同的参数如下：

- input_shape: 输入tensor的形状， 大多默认为244*244*3。一般下限为 32\*32或75\*75
- include_top: 网络的最后是否包含全链接层，默认为true
- weights: 预训练权值，默认为"imageNet", 即载入当前模型在imageNet数据集上预训练的权值，需要随机初始化可以设为None
- classes: 分类数，默认为1000. 修改参数需要include_top参数为true，且weights参数为None

各个网络的参数可以参见keras文档: https://keras.io/applications/